In [1]:
from numpy import * 

# 数据导入
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))
        dataMat.append(fltLine)
    return mat(dataMat)

# 计算距离
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA-vecB, 2)))

# 随机质心
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k, n)))
    for j in range(n):
        minJ = min(dataSet[:, j])
        rangeJ = float(max(dataSet[:, j]) - minJ)
        centroids[:, j] =  minJ + rangeJ * random.rand(k,1)
    return centroids


dataMat = loadDataSet('testSet.txt')
randCent(dataMat, 2)

matrix([[-0.5335604 , -2.82756406],
        [ 2.06636089, -2.55041043]])

In [22]:
"""
    k均值据类算法
    算法会创建k个质心，然后将每个点分配到最近的质心，在重新计算质心。该过程重复执行，知道质心不再改变
"""
def KMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m, 2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j, :], dataSet[i, :])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i, 0] != minIndex:
                clusterChanged = True
            clusterAssment[i, :] = minIndex, minDist**2
#         print(clusterAssment)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:, 0].A==cent)[0]]
            centroids[cent, :] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment


centroids, clusterAssment = KMeans(dataMat, 4)
centroids

matrix([[ 2.2728545 , -3.41806164],
        [ 1.73775604,  3.222066  ],
        [ 3.50531338, -1.201807  ],
        [-3.231108  , -0.45641674]])